In [1]:
import sys
sys.path.append('../../../src')
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'

In [2]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge, Molecule, ForceField, UpdaterMD, WithEnergyCell, set_global_units
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo

In [3]:
set_global_units('nm', 'kj/mol')
# context.set_context(mode=context.GRAPH_MODE, device_target='Ascend', device_id=0)
context.set_context(mode=context.GRAPH_MODE, device_id=0)

## 周期性边界条件

对于没有周期性的体系来说，我们用reduplicate和copy就可以实现系统的扩展，但是对于有周期性边界条件的系统而言，我们需要以box为单位进行复制，然后合成一个大的box。比如这里我们先用一个盒子的空间来定义一个水分子，然后把这个box向指定方向扩展5\*5\*5=125个盒子。也就得到了125个水分子，一共375个原子。

In [4]:
system = Molecule(template='water.tip3p.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])
system.repeat_box([5, 5, 5])

Molecule<>

In [5]:
print('The total number of atoms is: ', system.num_atoms)

The total number of atoms is:  375


## 能量极小化

跟前面的案例一样的流程，需要先做能量极小化。

In [6]:
potential = ForceField(system, parameters=['TIP3P'], use_pme=True)
opt = Adam(system.trainable_params(), 1e-3)
sim = WithEnergyCell(system, potential)
mini = Sponge(sim, optimizer=opt)
run_info = RunInfo(50)
mini.run(500, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-08-15 10:25:06
[MindSPONGE] Step: 0, E_pot: 11003.434
[MindSPONGE] Step: 50, E_pot: 9806.189
[MindSPONGE] Step: 100, E_pot: 9562.136
[MindSPONGE] Step: 150, E_pot: 9351.428
[MindSPONGE] Step: 200, E_pot: 9075.849
[MindSPONGE] Step: 250, E_pot: 8433.691
[MindSPONGE] Step: 300, E_pot: 7907.46
[MindSPONGE] Step: 350, E_pot: 7528.109
[MindSPONGE] Step: 400, E_pot: 7182.771
[MindSPONGE] Step: 450, E_pot: 6916.277
[MindSPONGE] Finished simulation at 2023-08-15 10:25:13
[MindSPONGE] Simulation time: 6.86 seconds.
--------------------------------------------------------------------------------


## NVT、NPT模拟

如果在UpdaterMD中指定temperature，就可以实现控温，也可以通过thermostat来配置控温算法，这里选择的是郎之万控温算法。如果不配置pressure的话，就是NVT模拟，不会对box进行调整。如果配置了pressure参数，就是一个NPT模拟过程，就会在模拟的过程中不断的变化box来控制压强。

In [7]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

nvt = UpdaterMD(
    system=system,
    time_step=1e-3,
    velocity=velocity,
    integrator='velocity_verlet',
    temperature=300,
    thermostat='langevin',
)
md = mini.change_optimizer(nvt)

run_info = RunInfo(200)
cb_h5md = WriteH5MD(system, 'tutorial_c03_nvt.h5md', save_freq=200, write_velocity=True, write_force=True)
md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 10:25:55
[MindSPONGE] Step: 0, E_pot: 6720.9424, E_kin: 1428.5167, E_tot: 8149.459, Temperature: 306.25858, Pressure: -1119.9005, Volume: 8.0
[MindSPONGE] Step: 200, E_pot: 7209.6904, E_kin: 1165.6301, E_tot: 8375.32, Temperature: 249.89856, Pressure: -95.48153, Volume: 8.0
[MindSPONGE] Step: 400, E_pot: 7137.53, E_kin: 1399.1545, E_tot: 8536.685, Temperature: 299.96368, Pressure: -2372.6106, Volume: 8.0
[MindSPONGE] Step: 600, E_pot: 7240.3726, E_kin: 1376.6489, E_tot: 8617.021, Temperature: 295.1387, Pressure: -1466.1366, Volume: 8.0
[MindSPONGE] Step: 800, E_pot: 7220.04, E_kin: 1449.6494, E_tot: 8669.689, Temperature: 310.78925, Pressure: -2747.711, Volume: 8.0
[MindSPONGE] Step: 1000, E_pot: 7111.4424, E_kin: 1447.9503, E_tot: 8559.393, Temperature: 310.42496, Pressure: -1074.3214, Volume: 8.0
[MindSPONGE] Step: 1200, E_pot: 6992.012, E_kin: 1443.3613, E_tot: 8435.373, Temperature: 309.44113, Pressure: -521.65826, Volume: 8.0
[MindSPON

In [8]:
npt = UpdaterMD(
    system=system,
    time_step=1e-3,
    velocity=velocity,
    integrator='velocity_verlet',
    temperature=300,
    pressure=1,
    thermostat='langevin',
)
md.change_optimizer(npt)

run_info = RunInfo(200)
cb_h5md = WriteH5MD(system, 'tutorial_c03_npt.h5md', save_freq=200, write_velocity=True, write_force=True)
md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 10:26:30
[MindSPONGE] Step: 0, E_pot: 7046.3394, E_kin: 1428.5167, E_tot: 8474.856, Temperature: 306.25858, Pressure: -1771.9061, Volume: 8.0
[MindSPONGE] Step: 200, E_pot: 7087.521, E_kin: 1439.8352, E_tot: 8527.356, Temperature: 308.68518, Pressure: 921.82355, Volume: 7.9263415
[MindSPONGE] Step: 400, E_pot: 7014.917, E_kin: 1370.981, E_tot: 8385.898, Temperature: 293.92355, Pressure: -1167.0834, Volume: 7.863459
[MindSPONGE] Step: 600, E_pot: 6992.448, E_kin: 1437.8881, E_tot: 8430.336, Temperature: 308.2677, Pressure: -1167.8602, Volume: 7.8318396
[MindSPONGE] Step: 800, E_pot: 7092.685, E_kin: 1295.2793, E_tot: 8387.965, Temperature: 277.69394, Pressure: -799.5288, Volume: 7.7849855
[MindSPONGE] Step: 1000, E_pot: 6919.3228, E_kin: 1484.2681, E_tot: 8403.591, Temperature: 318.2111, Pressure: -2241.4954, Volume: 7.7078714
[MindSPONGE] Step: 1200, E_pot: 6891.2686, E_kin: 1403.2451, E_tot: 8294.514, Temperature: 300.84064, Pressure: -101

In [9]:
print('The final pbc box is: {}'.format(system.pbc_box.asnumpy()))

The final pbc box is: [[1.9481812 1.9481812 1.9481812]]
